In [1]:
import cupy as cp
from scipy.spatial.transform import Rotation as R
import numpy as np
import copy
import time
import matplotlib.pyplot as plt
import open3d as o3d
import utm

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
o3d.core.Tensor(np.asarray([10,5,5]))

[10 5 5]
Tensor[shape={3}, stride={1}, Int64, CPU:0, 0x5d2c080]

In [ ]:
pcd_14 = o3d.io.read_point_cloud("/home/turin/Desktop/lizard_dataset_curated/2014/pcd14.pcd")
pcd_15 = o3d.io.read_point_cloud("/home/turin/Desktop/lizard_dataset_curated/2015/pcd15.pcd")
pcd_14_cp = copy.deepcopy(pcd_14)
pcd_15_cp = copy.deepcopy(pcd_15)
r = R.from_rotvec([np.pi / 9, 0, np.pi / 6])
rot_mat = r.as_matrix().reshape(3, 3)
pcd_14_cp.translate(np.asarray([1.12, 1.55, -1.1]))
pcd_14_cp.rotate(rot_mat, center=pcd_14_cp.get_center())
pcd_14_cp.scale(scale=1.1, center=pcd_14_cp.get_center())

In [4]:
#FAST GLOBAL TRANSFORMATION
start = time.time()
voxel_size = 0.1  # 0.05 means 5cm for this dataset
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(pcd_14_cp, pcd_15_cp,
    voxel_size)

# result_fast = execute_fast_global_registration(source_down, target_down,
#                                                source_fpfh, target_fpfh,
#                                                voxel_size)
# print("Fast global registration took %.3f sec.\n" % (time.time() - start))
# print(result_fast)
# result_fast.transformation
# draw_registration_result(source, target, result_fast.transformation)

result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down,
        target_down,
        source_fpfh,
        target_fpfh,
        mutual_filter=True,
        max_correspondence_distance=8.5*.1,
        estimation_method=o3d.pipelines.registration.
        TransformationEstimationPointToPoint(True),
        ransac_n=3,
        checkers=[
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                8.5*.1)
        ],
        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(1000000, 0.9999))
source.paint_uniform_color([1, 0, 0])
target.paint_uniform_color([0, 1, 0])
#o3d.visualization.draw([source.transform(result.transformation), target])

:: Load two point clouds with initial pose.
:: Downsample with a voxel size 0.100.
:: Estimate normal with search radius 0.200.
:: Compute FPFH feature with search radius 0.300.
:: Downsample with a voxel size 0.100.
:: Estimate normal with search radius 0.200.
:: Compute FPFH feature with search radius 0.300.


PointCloud with 42312642 points.

In [5]:
o3d.visualization.draw([source.transform(result.transformation), target])

[Open3D INFO] Window window_0 created.
[Open3D INFO] EGL headless mode enabled.
[Open3D INFO] ICE servers: {"stun:stun.l.google.com:19302", "turn:user:password@34.69.27.100:3478", "turn:user:password@34.69.27.100:3478?transport=tcp"}
[Open3D INFO] Set WEBRTC_STUN_SERVER environment variable add a customized WebRTC STUN server.
[Open3D INFO] WebRTC Jupyter handshake mode enabled.


KeyboardInterrupt: 

In [11]:
#target.transform(result.transformation)
print(result.transformation)
draw_registration_result(pcd_14_cp,pcd_14,result.transformation)

[[-9.36799159e-01  1.81132811e-01 -3.35130476e-02  8.35992386e+01]
 [ 1.51348057e-01  8.55860735e-01  3.95121566e-01  8.54880374e-01]
 [ 1.05004691e-01  3.82384873e-01 -8.68493355e-01 -3.17422027e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [ ]:
 ##PLY TO PCD
# source_to_pcd = o3d.io.read_point_cloud("/home/turin/Desktop/lizard_dataset_curated/old_dataset_coral/cur_cas_05m_20190401_dec7M.ply")
# o3d.io.write_point_cloud("/home/turin/Desktop/lizard_dataset_curated/source.pcd", source_to_pcd)
pcd_14 = o3d.io.read_point_cloud("/home/turin/Desktop/lizard_dataset_curated/2014/pcd14.pcd")
pcd_15 = o3d.io.read_point_cloud("/home/turin/Desktop/lizard_dataset_curated/2015/pcd15.pcd")
# pcd_16 = o3d.io.read_point_cloud("/home/turin/Desktop/lizard_dataset_curated/2016/pcd16.pcd")
pcd_14_cp = copy.deepcopy(pcd_14)
pcd_15_cp = copy.deepcopy(pcd_15)
# pcd_16_cp = copy.deepcopy(pcd_16)

In [ ]:
draw_registration_result(pcd_14_cp, pcd_15_cp, np.identity(4))

In [ ]:
dist_error = pcd_15_cp.compute_point_cloud_distance(pcd_14_cp)

In [ ]:
from scipy.spatial.transform import Rotation as R
# r = R.from_rotvec([np.pi/6,np.pi/6,np.pi/6])
r = R.from_rotvec([np.pi/9,0,np.pi/6])
rot_mat = r.as_matrix().reshape(3,3)
# trans_mat = np.asarray([0.25,0.91,0.77]).reshape(3,1)
# transform_mat = np.vstack((np.hstack((rot_mat,trans_mat)), [0,0,0,1]))#Transfor mat with trans and rot mat
# transform = np.matmul(transform_mat, np.diag(np.full(4,.75))) ## Scaling
# print(transform_mat)
pcd_14_cp = copy.deepcopy(pcd_14)
pcd_14_cp.translate(np.asarray([1.12,1.55,-1.1]))
pcd_14_cp.rotate(rot_mat, center = pcd_14.get_center())
pcd_14_cp.scale(scale = 1.5, center = pcd_14.get_center())
o3d.visualization.draw_geometries([pcd_14,pcd_14_cp])
#draw_registration_result(pcd_14, pcd_14_cp, trans_init)

In [ ]:
error_fast = []
for x in range(len(dist_error)):
    error_fast.append(dist_error[x])

In [ ]:
start = time.time()
hist, bin = np.histogram(error_fast, 1000)
#hist = cp.asarray(hist.get())
#bin = cp.asarray(bin.get())
plt.bar(bin[:-1] + np.diff(bin) / 2, hist, np.diff(bin))
# _ = plt.hist(error_fast, bins=1000)
plt.xlabel("Error in meters(for 1000 bins)")
plt.ylabel("number of cloud point")
plt.yscale("log")
plt.show()

plt.bar(bin[:-1] + np.diff(bin) / 2, hist, np.diff(bin))
# _ = plt.hist(error_fast, bins=1000)
plt.xlabel("Error in meters(for 1000 bins)")
plt.ylabel("number of cloud point")
plt.show()



print("took %.3f sec.\n" % (time.time() - start))

In [ ]:
dist_error = pcd_15_cp.compute_point_cloud_distance(pcd_14_cp)
error_fast = []
for x in range(len(dist_error)):
    error_fast.append(dist_error[x])
print(len(error_fast))
_ = plt.hist(error_fast, bins=1000)
plt.xlabel("Error in meters(for 1000 bins)")
plt.ylabel("number of cloud point")
plt.show()

In [ ]:
_ = plt.hist(error_fast, bins=1000)
plt.xlabel("Error in meters(for 1000 bins)")
plt.ylabel("number of cloud point in Log10 Scale")
plt.yscale("log")
plt.show()

In [ ]:
error_init = error_fast
max(error_init), min(error_init),sum(error_init)/len(error_init)

In [ ]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 3
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

def prepare_dataset(source, target, voxel_size):
    print(":: Load two point clouds with initial pose.")
    # trans_init = np.asarray([[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0],
    #                          [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    # source.transform(trans_init)
    draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

def execute_fast_global_registration(source_down, target_down, source_fpfh,
                                     target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: Apply fast global registration with distance threshold %.3f" \
            % distance_threshold)
    result = o3d.pipelines.registration.registration_fgr_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh,
        o3d.pipelines.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=distance_threshold))
    return result

In [ ]:
#FAST GLOBAL TRANSFORMATION
start = time.time()
voxel_size = 0.1  # 0.05 means 5cm for this dataset
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(pcd_14_cp, pcd_15_cp,
    voxel_size)

result_fast = execute_fast_global_registration(source_down, target_down,
                                               source_fpfh, target_fpfh,
                                               voxel_size)
print("Fast global registration took %.3f sec.\n" % (time.time() - start))
print(result_fast)
result_fast.transformation
draw_registration_result(source, target, result_fast.transformation)

In [ ]:
source_fast = copy.deepcopy(source).transform(result_fast.transformation)
dif = target.compute_point_cloud_distance(source_fast)

In [ ]:
error_fast = []
for x in range(len(dif)):
    error_fast.append(dif[x])
print(len(error_fast))

In [ ]:
_ = plt.hist(error_fast, bins=1000)
plt.xlabel("Error in meters(for 1000 bins)")
plt.ylabel("number of cloud point in log 10 scale")
plt.yscale("log")
plt.show()

In [ ]:
_[0]

In [ ]:
min(error_fast), max(error_fast), sum(error_fast)/len(error_fast)

In [ ]:
#Registration ICP
voxel_size = .1
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(pcd_14_cp, pcd_15_cp,
    voxel_size)
treg = o3d.pipelines.registration
max_correspondence_distance = 0.1
trans_init = np.identity(4)#np.asarray(result.transformation, dtype=np.float64)
#mu, sigma = 0, 0.1  # mean and standard deviation
loss = o3d.pipelines.registration.TukeyLoss(0.1)
estimation = treg.TransformationEstimationPointToPoint(True)
# Convergence-Criteria for Vanilla ICP
criteria = treg.ICPConvergenceCriteria(relative_fitness=0.0000001,
                                       relative_rmse=0.000001,
                                       max_iteration=20)
# Save iteration wise `fitness`, `inlier_rmse`, etc. to analyse and tune result.
save_loss_log = True
registration_icp = treg.registration_icp(source, target, max_correspondence_distance,
                            trans_init, estimation, criteria)

print("Inlier Fitness: ", registration_icp.fitness)
print("Inlier RMSE: ", registration_icp.inlier_rmse)

draw_registration_result(source, target, registration_icp.transformation)

In [ ]:
source = o3d.t.io.read_point_cloud("/home/turin/Desktop/lizard_dataset_curated/2014/pcd14.pcd")
target = o3d.t.io.read_point_cloud("/home/turin/Desktop/lizard_dataset_curated/2015/pcd15.pcd")

In [ ]:
source = o3d.t.geometry.PointCloud()
target = o3d.t.geometry.PointCloud()
source = source.from_legacy(pcd_14_cp)
target = target.from_legacy(pcd_15_cp)

In [ ]:
init_trans = o3d.core.Tensor(result.transformation)
print(init_trans)

In [ ]:
treg = o3d.t.pipelines.registration
voxel_sizes = o3d.utility.DoubleVector([0.1, 0.08, 0.07])
criteria_list = [
    treg.ICPConvergenceCriteria(relative_fitness=0.000001,
                                relative_rmse=0.000001,
                                max_iteration=200),
    treg.ICPConvergenceCriteria(0.0000001, 0.0000001, 20),
    treg.ICPConvergenceCriteria(0.00000001, 0.00000001, 10)
]
max_correspondence_distances = o3d.utility.DoubleVector([.7, 0.7, 0.7])
estimation = treg.TransformationEstimationPointToPoint(True)

# Save iteration wise `fitness`, `inlier_rmse`, etc. to analyse and tune result.
callback_after_iteration = lambda loss_log_map : print("Iteration Index: {}, Scale Index: {}, Scale Iteration Index: {}, Fitness: {}, Inlier RMSE: {},".format(
    loss_log_map["iteration_index"].item(),
    loss_log_map["scale_index"].item(),
    loss_log_map["scale_iteration_index"].item(),
    loss_log_map["fitness"].item(),
    loss_log_map["inlier_rmse"].item()))

init_source_to_target = o3d.core.Tensor.eye(4, o3d.core.Dtype.Float32)
registration_ms_icp = treg. multi_scale_icp(source, target, voxel_sizes,
                                           criteria_list,
                                           max_correspondence_distances,init_source_to_target,
                                            estimation, callback_after_iteration)

print("Inlier Fitness: ", registration_ms_icp.fitness)
print("Inlier RMSE: ", registration_ms_icp.inlier_rmse)
draw_registration_result(source, target, registration_ms_icp.transformation)

In [ ]:
draw_registration_result(source.to_legacy(), target.to_legacy(), registration_ms_icp.transformation.numpy())

In [ ]:
Inlier Fitness:  0.9694830324971911
Inlier RMSE:  0.14174411476420193

In [ ]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
source_leg = source.to_legacy()
target_leg = target.to_legacy()
reg = registration_ms_icp.transformation.numpy()
draw_registration_result(source_leg, target_leg, reg)

In [ ]:
source_corr = copy.deepcopy(source_leg).transform(reg)

In [ ]:
dif = target_leg.compute_point_cloud_distance(source_corr)

In [ ]:
dist_error = dif
error = []
for x in range(len(dist_error)):
    error.append(dist_error[x])
print(len(error))
_ = plt.hist(error, bins=100)
plt.xlabel("Error in meters(for 100 bins)")
plt.ylabel("number of cloud point")
plt.show()

In [ ]:
max(error), min(error), sum(error)/len(error), np.log(max(error)),np.log(sum(error)/len(error)) ,np.log(min(error))

In [ ]:
_ = plt.hist(error, bins=1000)
plt.xlabel("Error in meters(for 1000 bins)")
plt.ylabel("number of cloud point in Log10 scale")
plt.yscale("log")
plt.show()

In [ ]:
reg

In [ ]:
pcd_14.get_axis_aligned_bounding_box()

In [ ]:
1000000*np.log(10)

In [ ]:
#
#
#
#
#
#

In [ ]:
#
#Alignment using transformation matrix
transformation_mat = registration_icp.transformation
source_array = np.asarray(source.points)
target_array = np.asarray(target.points)
transformation_mat, source_array.shape
source_temp = np.ones([4,len(source_array)], dtype= np.float64)
target_temp = np.ones([4,len(target_array)], dtype= np.float64)
for i in range(len(source_array)):
    source_temp[0:3,i] = source_array[i,:]
for i in range(len(target_array)):
    target_temp[0:3,i] = target_array[i,:]
new_coord = np.matmul(transformation_mat, source_temp)
source_array_transformed = new_coord.T

In [ ]:
registration_icp.transformation

In [ ]:
#Error(mean root square over x,y,z axis) after alignment
error = []
for idx in range(len(target_array)):
    er = (target_array[idx,0] - source_array_transformed [idx,0])**2 + (target_array[idx,1] - source_array_transformed[idx,1])**2 + (target_array[idx,2] - source_array_transformed[idx,2])**2
    er = np.sqrt(er)
    error.append(er)
error = np.asarray(error)
#Error Distribution
print(error.min(),error.max())
_ = plt.hist(error, bins='auto')
plt.show()

In [ ]:
oct_tree1 = o3d.geometry.Octree(max_depth=8)
oct_tree2 = o3d.geometry.Octree(max_depth=8)
source_array = o3d.geometry.PointCloud()
source_array.points = o3d.utility.Vector3dVector(np.asarray(source_array_transformed))
oct_tree1.convert_from_point_cloud(source_array)
oct_tree2.convert_from_point_cloud(target)



In [ ]:
#Error color mapping and Error visualization
def rgb(minimum, maximum, value):
    minimum, maximum = float(minimum), float(maximum)
    ratio = 2 * (value-minimum) / (maximum - minimum)
    b = int(max(0, 255*(1 - ratio)))
    r = int(max(0, 255*(ratio - 1)))
    g = 255 - b - r
    return r, g, b

color = np.zeros((1000,3))
from time import perf_counter
start = perf_counter()
for idx in range(1000):
    r,g,b = rgb(error.min(), error.max(), error[idx])
    color[idx,0] = r
    color[idx,1] = g
    color[idx,2] = b
end = perf_counter()
print(end-start)
# color = np.zeros((len(source_array_transformed),3))
# for idx in range(len(source_array_transformed)):
#     r,g,b = rgb(error.min(), error.max(), error[idx])
#     color[idx,0] = r
#     color[idx,1] = g
#     color[idx,2] = b
#     print(idx)
# print(color.shape)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(source_array_transformed[:1000,:3])
pcd.colors = o3d.utility.Vector3dVector(color)
o3d.io.write_point_cloud("error.ply", pcd)
pcd_load = o3d.io.read_point_cloud("error.ply")
o3d.visualization.draw_geometries([pcd_load])

In [ ]:
print(source_array.shape,target_array_transformed)

In [1]:
import pypdf
pypdf.